# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 9:38AM,238249,10,0085717809,ISDIN Corporation,Released
1,May 27 2022 9:37AM,238247,10,0085715132,ISDIN Corporation,Released
2,May 27 2022 9:37AM,238247,10,0085715133,ISDIN Corporation,Released
3,May 27 2022 9:37AM,238247,10,0085715146,ISDIN Corporation,Released
4,May 27 2022 9:37AM,238247,10,0085715154,ISDIN Corporation,Released
5,May 27 2022 9:37AM,238247,10,0085715156,ISDIN Corporation,Released
6,May 27 2022 9:37AM,238247,10,0085715155,ISDIN Corporation,Released
7,May 27 2022 9:37AM,238247,10,0085715172,ISDIN Corporation,Released
8,May 27 2022 9:37AM,238247,10,0085715179,ISDIN Corporation,Released
9,May 27 2022 9:37AM,238247,10,0085715180,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238241,Released,7
34,238243,Released,27
35,238244,Executing,1
36,238247,Released,31
37,238249,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238241,NaN,NaN,7.0
238243,NaN,NaN,27.0
238244,NaN,1.0,NaN
238247,NaN,NaN,31.0
238249,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
238153,0.0,1.0,0.0
238159,0.0,0.0,1.0
238163,0.0,0.0,1.0
238165,0.0,2.0,7.0
238168,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
238153,0.0,1,0
238159,0.0,0,1
238163,0.0,0,1
238165,0.0,2,7
238168,0.0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,238153,0.0,1,0
1,238159,0.0,0,1
2,238163,0.0,0,1
3,238165,0.0,2,7
4,238168,0.0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,238153,0.0,1,
1,238159,0.0,,1
2,238163,0.0,,1
3,238165,0.0,2,7
4,238168,0.0,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation
32,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc."
33,May 27 2022 9:16AM,238243,10,ISDIN Corporation
60,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC"
67,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated"
68,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC"
70,May 27 2022 8:46AM,238237,10,Beach Products Inc
85,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc."
90,May 27 2022 8:43AM,238236,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation,0.0,,1
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation,0.0,,31
2,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc.",0.0,1,
3,May 27 2022 9:16AM,238243,10,ISDIN Corporation,0.0,,27
4,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",0.0,,7
5,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",0.0,,1
6,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",0.0,,2
7,May 27 2022 8:46AM,238237,10,Beach Products Inc,0.0,5,10
8,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc.",0.0,5,
9,May 27 2022 8:43AM,238236,10,Emerginnova,0.0,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1,
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31,
2,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc.",,1
3,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
4,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7,
5,May 27 2022 8:49AM,238240,10,"Senseonics, Incorporated",1,
6,May 27 2022 8:48AM,238239,15,"Carwin Pharmaceutical Associates, LLC",2,
7,May 27 2022 8:46AM,238237,10,Beach Products Inc,10,5
8,May 27 2022 8:46AM,238234,15,"Person & Covey, Inc.",,5
9,May 27 2022 8:43AM,238236,10,Emerginnova,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1,
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31,
2,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc.",,1
3,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27,
4,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1.0,NaN
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31.0,NaN
2,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc.",NaN,1.0
3,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27.0,NaN
4,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 27 2022 9:38AM,238249,10,ISDIN Corporation,1.0,0.0
1,May 27 2022 9:37AM,238247,10,ISDIN Corporation,31.0,0.0
2,May 27 2022 9:33AM,238244,19,"GCH Granules USA, Inc.",0.0,1.0
3,May 27 2022 9:16AM,238243,10,ISDIN Corporation,27.0,0.0
4,May 27 2022 8:54AM,238241,15,"Virtus Pharmaceuticals, LLC",7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3096828,85.0,5.0
12,238210,1.0,0.0
15,952901,35.0,5.0
16,952825,10.0,1.0
18,238159,1.0,0.0
19,1191039,4.0,22.0
20,952687,15.0,4.0
21,238163,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3096828,85.0,5.0
1,12,238210,1.0,0.0
2,15,952901,35.0,5.0
3,16,952825,10.0,1.0
4,18,238159,1.0,0.0
5,19,1191039,4.0,22.0
6,20,952687,15.0,4.0
7,21,238163,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,85.0,5.0
1,12,1.0,0.0
2,15,35.0,5.0
3,16,10.0,1.0
4,18,1.0,0.0
5,19,4.0,22.0
6,20,15.0,4.0
7,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,12,Released,1.0
2,15,Released,35.0
3,16,Released,10.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Executing,5.0,0.0,5.0,1.0,0.0,22.0,4.0,0.0
Released,85.0,1.0,35.0,10.0,1.0,4.0,15.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Executing,5.0,0.0,5.0,1.0,0.0,22.0,4.0,0.0
1,Released,85.0,1.0,35.0,10.0,1.0,4.0,15.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Executing,5.0,0.0,5.0,1.0,0.0,22.0,4.0,0.0
1,Released,85.0,1.0,35.0,10.0,1.0,4.0,15.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()